In [1]:
import pyodbc
import pandas as pd
import shutil
import re


def copy_database(original_path, temp_path):
    shutil.copyfile(original_path, temp_path)

def export_access_to_dataframes(database_path):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Get a list of all tables in the database
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]

    # Loop through the tables and load each into a DataFrame
    for table in tables:
        query = f'SELECT * FROM [{table}]'
        df = pd.read_sql(query, conn)
        globals()[f'tbl_{table}'] = df  # Create a global variable with the table name

    # Close the connection
    conn.close()

def update_access_table(database_path, table_name, df):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Clear the existing table
    cursor.execute(f'DELETE FROM [{table_name}]')
    conn.commit()

    # Insert the updated data
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        placeholders = ', '.join(['?' for _ in row])
        values = tuple(row)
        sql = f'INSERT INTO [{table_name}] ({columns}) VALUES ({placeholders})'
        cursor.execute(sql, values)
    conn.commit()

    # Close the connection
    conn.close()

# Paths
original_database_path = 'C:\\digidure\\CLERUS_v05082024.accdb'
temp_database_path = 'C:\\digidure\\CLERUS_v05082024-2.accdb'

# Copy the database
copy_database(original_database_path, temp_database_path)

# Export data from the copied database
export_access_to_dataframes(temp_database_path)

c:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are no

In [2]:
# Function to append the data to access database
def append_to_access_table(df, database_path, table_name):
    """
    Appends a DataFrame to a table in an Access database.

    Parameters:
    df (pandas.DataFrame): The DataFrame to append.
    database_path (str): The path to the Access database file.
    table_name (str): The name of the table to append the data to.
    """
    # Connection string for Access database
    connection_string = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Connect to the Access database
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    # Append DataFrame to the Access table
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        values = ', '.join(['?' for _ in row])
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({values})'
        cursor.execute(sql, tuple(row))

    conn.commit()
    cursor.close()
    conn.close()

In [3]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# All the individuals that could be matched with DRC are already curated and processed. The next step is to extract individuals from DM that are not in DRC.
# When a link between DM and DRC was found, a value for clerus_id or new_clerus_id was produced. Thus to isolate those that have not been matched we create
# a subselection existing of rows where these fields are empty.
filtered_only_DM = tbl_999_Dm_all_drc_match[tbl_999_Dm_all_drc_match['clerus_id'].isna() & tbl_999_Dm_all_drc_match['new_clerus_id'].isna()]


In [5]:
# Since we want to link the individuals to clerus we want to use the individual_id lateron as a new clerus_id. To avoid overlap in numbers we added 9000000 to every id. The individual ids have been created through 2_1_check_links_DRC_DM.ipynb
filtered_only_DM["individual_id"] = filtered_only_DM["individual_id"] + 9000000

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\3862527704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_only_DM["individual_id"] = filtered_only_DM["individual_id"] + 9000000


In [6]:
filtered_only_DM.head(15)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,individual_id,clerus_id,new_clerus_id
0,35651,1,ZH,None,Oudewater,None,(Hekendorp) Brunner; L.,hulppredikant Oirschot,None,None,1867,Lobith,None,None,1868,None,9013955,NaN,NaN
1,45546,2,GE,None,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,None,9013956,NaN,NaN
2,7179,3,NB,None,Breda,None,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,None,9013962,NaN,NaN
3,21505,3,GR,None,Hornhuizen,None,Aa; dr. P.J.B.K. Simon van der,kandidaat,None,None,1862,Goutum,None,None,1866,None,9013959,NaN,NaN
4,36887,3,NH,None,Purmerend,None,Aa; dr. P.J.B.K. Simon van der,Goutum,None,None,1886,Winterswijk,None,None,1892,None,9013960,NaN,NaN
5,50146,3,GE,None,Winterswijk,None,Aa; dr. P.J.B.K. Simon van der,Purmerend,None,None,1892,Breda,None,None,1893,None,9013960,NaN,NaN
6,15179,3,FR,None,Goutum,None,Aa; dr. P.J.B.K. van der,Kloosterburen,None,None,1866,Purmerend,None,None,1886,None,9013961,NaN,NaN
7,7204,3,NB,None,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,None,9013958,NaN,NaN
8,19779,4,GE,None,Herwijnen,None,Aa; W.P. van der,kandidaat,24,december,1989,None,None,None,None,None,9013957,NaN,NaN
9,35941,5,OV,None,Overdinkel,None,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,None,9013963,NaN,NaN


In [7]:
#From this dataset we will be extracting all kind of information to align with the Clerus database structure.

In [8]:
#First we extract information for every individual which we use to fill in the table 01_clerus_bio. These are the names of the individual, the year of death and remarks (Bijzonderheden).
# since we are not interested in all the other fields we make a subselection

DM_tbl_bio_1 = filtered_only_DM[['pid', 'predikant', "vertrek naar of vanwege", "jaar vertrek", "Bijzonderheden", "individual_id"]]

In [9]:
# Extract Year of Death
# After analysing the data in DM it appears that information about someone's year of death can be found in the field "Bijzonderheden", but also in the field "vertrek naar of vanwege".
# To extract this information and to save it in a new field is done in two steps.

# first we extract year from Bijzonderheden where it contains the string "overl." or "overleden"
def extract_year(bijzonderheden):
    if isinstance(bijzonderheden, str):
        match = re.search(r'(overleden|overl\.)\s*(\d{4})', bijzonderheden, re.IGNORECASE)
        if match:
            return int(match.group(2))
    return None

DM_tbl_bio_1['death_year'] = DM_tbl_bio_1['Bijzonderheden'].apply(extract_year)

# Second we update 'death_year' where it is None, using 'jaar_vertrek' if 'vertrek_naar_of_vanwege' contains 'overleden' or 'overl.'
DM_tbl_bio_1['death_year'] = DM_tbl_bio_1.apply(
    lambda row: row['jaar vertrek']
    if pd.isnull(row['death_year']) and
       isinstance(row['vertrek naar of vanwege'], str) and
       ('overleden' in row['vertrek naar of vanwege'].lower() or 'overl.' in row['vertrek naar of vanwege'].lower())
    else row['death_year'],
    axis=1
)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\2993844103.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_1['death_year'] = DM_tbl_bio_1['Bijzonderheden'].apply(extract_year)
C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\2993844103.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_1['death_year'] = DM_tbl_bio_1.apply(


In [10]:
DM_tbl_bio_1.head(1000)

,pid,predikant,vertrek naar of vanwege,jaar vertrek,Bijzonderheden,individual_id,death_year
0,35651,(Hekendorp) Brunner; L.,Lobith,1868,None,9013955,NaN
1,45546,(Hilten van-)Matthijsen;mevrouw L.,Apeldoorn,1994,None,9013956,NaN
2,7179,Aa; dr Pieter Jan Baptist Karel Simon van der,emeritaat,1904,None,9013962,NaN
3,21505,Aa; dr. P.J.B.K. Simon van der,Goutum,1866,None,9013959,NaN
4,36887,Aa; dr. P.J.B.K. Simon van der,Winterswijk,1892,None,9013960,NaN
5,50146,Aa; dr. P.J.B.K. Simon van der,Breda,1893,None,9013960,NaN
6,15179,Aa; dr. P.J.B.K. van der,Purmerend,1886,None,9013961,NaN
7,7204,Aa; dr. P.J.B.K.S. van der,emeritaat,1904,None,9013958,NaN
8,19779,Aa; W.P. van der,None,None,None,9013957,NaN
9,35941,Aaij; J.,emeritaat,1986,None,9013963,NaN


In [11]:
# Next we need to make another subselection of the data to get only the name, clerus_id, year of death and remarks.
DM_tbl_bio_2 = DM_tbl_bio_1[['predikant', "Bijzonderheden", "individual_id", "death_year"]]

In [12]:
DM_tbl_bio_2['death_year'] = DM_tbl_bio_2['death_year'].fillna(0).astype(int)
print(DM_tbl_bio_2.dtypes)

predikant         object
Bijzonderheden    object
individual_id      int64
death_year         int32
dtype: object


C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\1663001344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_2['death_year'] = DM_tbl_bio_2['death_year'].fillna(0).astype(int)


In [13]:
# Now we remove all duplicates that remain based on the inidivual_id and prioritize on whether "year_death" has a value.

DM_tbl_bio_2 = DM_tbl_bio_2.sort_values(by='death_year', ascending=False, na_position='last')



In [14]:
DM_tbl_bio_2.describe()

,individual_id,death_year
count,2.946300e+04,29463.000000
mean,9.022581e+06,168.277399
std,5.430808e+03,539.533276
min,9.000025e+06,0.000000
25%,9.018184e+06,0.000000
50%,9.022633e+06,0.000000
75%,9.027134e+06,0.000000
max,9.031653e+06,2005.000000


In [15]:
# Drop duplicates based on 'individual_id', keeping the first occurrence
DM_tbl_bio_unique = DM_tbl_bio_2.drop_duplicates(subset='individual_id', keep='first')

In [16]:
DM_tbl_bio_unique[['surname', 'first_name']] = DM_tbl_bio_unique['predikant'].str.split(';', expand=True)
DM_tbl_bio_unique.drop(columns=['predikant'], inplace=True)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\550979927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_unique[['surname', 'first_name']] = DM_tbl_bio_unique['predikant'].str.split(';', expand=True)
C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\550979927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_unique[['surname', 'first_name']] = DM_tbl_bio_unique['predikant'].str.split(';', expand=True)
C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25

In [17]:
DM_tbl_bio_unique.rename(columns={'Bijzonderheden': 'remarks', 'individual_id': 'clerus_id'}, inplace=True)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\2943604984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_unique.rename(columns={'Bijzonderheden': 'remarks', 'individual_id': 'clerus_id'}, inplace=True)


In [18]:
DM_tbl_bio_unique.head(10)

,remarks,clerus_id,death_year,surname,first_name
785,None,9014158,2005,Alma,G.S.
49910,None,9030309,2000,Vrijlandt,M.A.
44667,Parttime,9028633,1999,Storm,G. van der
29816,None,9023861,1998,Magedans,F.C.
13378,None,9018125,1996,Emmen,L.E.
26048,None,9022453,1995,Korporaaj,G.K.
48862,None,9029961,1994,Vlastuin,J.
49838,(overleden1994),9030294,1994,Vries,W. de
48851,None,9029960,1994,Vlasblom,W.A.
22683,None,9021135,1994,Jansen,M.


In [19]:
# Append data to database

table_name = '01_clerus_bio'
append_to_access_table(DM_tbl_bio_unique, temp_database_path, table_name)

In [20]:
# Get roles seperate out of the dataframe where roles from individuals that are only present in DM are used. (i.e. filtered_only_DM)
# Provincie [role_province], Classis [role_classis], Gemeente [role_place], clerus_id, pid(as backup), jaar intrede [year_start], jaar vertrek [year_end], dag intrede - maand intrede - jaar intrede [role_start_accuracy], dag vertrek - maand vertrek - jaar vertrek [role_end_accuracy]

filtered_only_DM.head()

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,individual_id,clerus_id,new_clerus_id
0,35651,1,ZH,None,Oudewater,None,(Hekendorp) Brunner; L.,hulppredikant Oirschot,None,None,1867,Lobith,None,None,1868,None,9013955,NaN,NaN
1,45546,2,GE,None,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,None,9013956,NaN,NaN
2,7179,3,NB,None,Breda,None,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,None,9013962,NaN,NaN
3,21505,3,GR,None,Hornhuizen,None,Aa; dr. P.J.B.K. Simon van der,kandidaat,None,None,1862,Goutum,None,None,1866,None,9013959,NaN,NaN
4,36887,3,NH,None,Purmerend,None,Aa; dr. P.J.B.K. Simon van der,Goutum,None,None,1886,Winterswijk,None,None,1892,None,9013960,NaN,NaN


In [21]:

role_dm = filtered_only_DM

In [22]:
# from the day, month and year create a date field which we will fill in to start_date_accuracy and end_date_accuracy
role_dm['role_start_date_exact'] = role_dm['dag intrede'].astype(str) + " " + role_dm['maand intrede'] + " " + role_dm['jaar intrede'].astype(str)
role_dm['role_end_date_exact'] = role_dm['dag vertrek'].astype(str) + " " + role_dm['maand vertrek'] + " " + role_dm['jaar vertrek'].astype(str)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\3336807610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_dm['role_start_date_exact'] = role_dm['dag intrede'].astype(str) + " " + role_dm['maand intrede'] + " " + role_dm['jaar intrede'].astype(str)
C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\3336807610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_dm['role_end_date_exact'] = role_dm['dag vertrek'].astype(str) + " " + role_dm['maand vertrek'] + " " + role_dm['j

In [23]:
role_dm.drop(columns=['ind_id','clerus_id', 'new_clerus_id', 'dag vertrek', 'maand vertrek', 'dag intrede', 'maand intrede', 'vertrek naar of vanwege', 'Herkomst', 'predikant'], inplace=True)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\1719675852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_dm.drop(columns=['ind_id','clerus_id', 'new_clerus_id', 'dag vertrek', 'maand vertrek', 'dag intrede', 'maand intrede', 'vertrek naar of vanwege', 'Herkomst', 'predikant'], inplace=True)


In [24]:
role_dm.rename(columns={'Bijzonderheden': 'role_remarks',
                        'individual_id': 'clerus_id',
                        'pid':'dm_id',
                        'provincie': 'role_province',
                        'classis': 'role_classis',
                        'gemeente': 'role_place',
                        'wijk': 'role_parish',
                        'jaar intrede': 'role_start_year',
                        'jaar vertrek': 'role_end_year'	}, inplace=True)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\2436905200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_dm.rename(columns={'Bijzonderheden': 'role_remarks',


In [25]:
role_dm['role_type'] = "predikant"

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\2014117642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_dm['role_type'] = "predikant"


In [26]:
role_dm['role_end_year'] = role_dm['role_end_year'].fillna(0).astype(int)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_25904\2373603986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_dm['role_end_year'] = role_dm['role_end_year'].fillna(0).astype(int)


In [27]:
role_dm.head()

,dm_id,role_province,role_classis,role_place,role_parish,role_start_year,role_end_year,role_remarks,clerus_id,role_start_date_exact,role_end_date_exact,role_type
0,35651,ZH,None,Oudewater,None,1867,1868,None,9013955,NaN,NaN,predikant
1,45546,GE,None,Velp,2. Oude Jan,1986,1994,None,9013956,2 maart 1986,23 oktober 1994,predikant
2,7179,NB,None,Breda,None,1893,1904,None,9013962,26 februari 1893,1 mei 1904,predikant
3,21505,GR,None,Hornhuizen,None,1862,1866,None,9013959,NaN,NaN,predikant
4,36887,NH,None,Purmerend,None,1886,1892,None,9013960,NaN,NaN,predikant


In [28]:
print(role_dm.dtypes)

dm_id                     int64
role_province            object
role_classis             object
role_place               object
role_parish              object
role_start_year           int64
role_end_year             int32
role_remarks             object
clerus_id                 int64
role_start_date_exact    object
role_end_date_exact      object
role_type                object
dtype: object


In [29]:
table_name = '12_clerus_role'
append_to_access_table(role_dm, temp_database_path, table_name)